### Нарушение согласования

Предыдущая попытка в задаче согласования рода - предсказывать род и сравнивать с ответом pymorphy

Недостаток - большое количество ошибочных предсказаний в словах, таких как "морской". 
Морской порт и цвет морской волны. Род различается, написание - нет.

Попробуем другой подход - предсказание окончаний.

Начнем с поиска ошибки согласования прилагательных

3 типа:
* новЫЙ
* хорошИЙ
* большОЙ

### 1. Построим бандл, сбалансированный по длине предложения и по корпусу.

In [2]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.corpus.corpus_reader import read_data

from yo_fluq_ds import Queryable, Query, fluq

from typing import List, Union
from pathlib import Path

import math
import pandas as pd

In [3]:
CORPUS_NAMES = [
    "books.base.zip", 
    "pub.base.zip"
]
#TODO: add smth else?

CORPUS_LIST = [Loc.corpus_path/corpus_name for corpus_name in CORPUS_NAMES]

In [4]:
CORPUS_LIST

[PosixPath('/home/serhio/Data/1Education/grammar-spring/s2/grammar_ru/data-cache/corpus/books.base.zip'),
 PosixPath('/home/serhio/Data/1Education/grammar-spring/s2/grammar_ru/data-cache/corpus/pub.base.zip')]

In [5]:
def get_sentences_count(corpus_path):
    return sum([
        len(frame.groupby("sentence_id")) 
        for frame in read_data(corpus_path)
    ])

In [6]:
for i, corpus_name in enumerate(CORPUS_NAMES):
    print(f"{corpus_name} = {get_sentences_count(CORPUS_LIST[i])} sentences")

/tmp/ipykernel_12481/3628676302.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(corpus_path)
100%|██████████| 2533/2533 [00:29<00:00, 84.56it/s] 
/tmp/ipykernel_12481/3628676302.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(corpus_path)


books.base.zip = 903668 sentences


100%|██████████| 66883/66883 [08:15<00:00, 135.01it/s]

pub.base.zip = 1472084 sentences


In [7]:
LOG_BASE = math.e
BUCKET_PATH = Loc.corpus_path/"prepare/buckets/buckets.parquet"

In [8]:
BucketCorpusBalancer.build_buckets_frame(CORPUS_LIST, BUCKET_PATH, LOG_BASE)

100%|██████████| 69416/69416 [39:07<00:00, 29.57it/s]  


In [9]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
bucket,,
books.base.zip/0,"[2105037, 2635119, 2635122, 2930082, 2930089, ...",129
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
books.base.zip/5,"[189579, 784022, 2391783, 5783820, 5998276, 67...",65
pub.base.zip/0,"[51423, 71661, 461963, 1207024, 1404782, 25887...",18308
pub.base.zip/1,"[24, 33, 34, 40, 52, 53, 64, 11030, 31091, 311...",60391
pub.base.zip/2,"[3, 5, 6, 8, 10, 14, 18, 21, 25, 26, 28, 30, 3...",415067


In [10]:
BUCKET_NUMBERS = [1, 2, 3, 4]
BUCKET_LIMIT = 60_000 #Note: забили на то что в books.base.zip/4 всего 23К предложений.
# некторый дисбаланс все-же остался

In [11]:
BucketCorpusBalancer.filter_buckets_by_bucket_numbers(BUCKET_PATH, BUCKET_NUMBERS)

In [12]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
pub.base.zip/1,"[24, 33, 34, 40, 52, 53, 64, 11030, 31091, 311...",60391
pub.base.zip/2,"[3, 5, 6, 8, 10, 14, 18, 21, 25, 26, 28, 30, 3...",415067
pub.base.zip/3,"[0, 1, 2, 4, 7, 9, 11, 13, 15, 16, 17, 19, 20,...",820561
pub.base.zip/4,"[12, 31092, 132279, 132283, 132285, 132289, 13...",156624


In [13]:
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/books&pub_60K_balanced.zip"

In [16]:
from tg.common import Logger
def balancing() -> None:
    balancer = BucketCorpusBalancer(
        buckets = pd.read_parquet(BUCKET_PATH), 
        log_base = LOG_BASE,
        bucket_limit = BUCKET_LIMIT,
    )

    CorpusBuilder.transfuse_corpus(
        sources = CORPUS_LIST,
        destination = BALANCED_PATH,
        selector = balancer
    )

In [20]:
balancing()

In [19]:
from collections import defaultdict
lengths = defaultdict(int)
for frame in read_data(BALANCED_PATH):
    for corpus_name in CORPUS_NAMES:
        lengths[corpus_name] += len(frame[frame.original_corpus_id == corpus_name].groupby("sentence_id"))

lengths

/tmp/ipykernel_12481/1361944034.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(BALANCED_PATH):
100%|██████████| 147/147 [00:11<00:00, 12.57it/s]


defaultdict(int, {'books.base.zip': 203177, 'pub.base.zip': 240000})